In [1]:
# Import libraries and needed classes

import numpy as np
import sys
import os

from Code.RNN.layers.sigmoid import SigmoidActivation
from Code.RNN.functions.metrics import mse, accuracy
from Code.RNN.networks.recorrent_neural_network import RecorrentNeuralNetwork
from Code.RNN.functions.mse import MeanSquaredError
from Code.RNN.layers.rnn import RNN
from Code.RNN.layers.dense import DenseLayer
from Code.RNN.optimizations.retained_gradient import RetGradient
from Code.DNN.optimizations.l2_reg import L2Reg
from Code.RNN.functions.bce import BinaryCrossEntropy
from Code.RNN.layers.relu import ReLUActivation

from Code.utils.dataset import Dataset
import Code.utils.store_model as store_model

In [2]:
# Set random seed to control randomness

np.random.seed(42)

In [3]:
# Read datasets

dataset = Dataset('../../Dataset/DatasetsGerados/dataset_training_input.csv',
                      '../../Dataset/DatasetsGerados/dataset_training_output.csv',
                      '../../Dataset/DatasetsGerados/dataset_validation_input.csv',
                      '../../Dataset/DatasetsGerados/dataset_validation_output.csv',
                      '../../Dataset/DatasetsGerados/dataset_test_input.csv',
                      '../../Dataset/DatasetsGerados/dataset_test_output.csv')

X_train, y_train, X_validation, y_validation, X_test, y_test, ids = dataset.get_dataset_embedding('Text', 'Label', sep='\t', rem_punctuation=False)

In [4]:
# Build network

batch_size = 8

# network
optimizer = RetGradient(learning_rate=0.001, momentum=0.90)
loss = BinaryCrossEntropy()

regulator = L2Reg(l2_val=0.01)
net = RecorrentNeuralNetwork(epochs=10, batch_size=batch_size, optimizer=optimizer, regulator=regulator, verbose=True, loss=loss,
                    metric=accuracy, patience=-1, min_delta=0.001)

net.add(RNN(10, input_shape=(X_train.shape[1], X_train.shape[2])))
net.add(ReLUActivation())

net.add(DenseLayer(1))
net.add(SigmoidActivation())

In [5]:
# Train network

net.fit(X_train, y_train, X_val=X_validation, y_val=y_validation)

# Plot learning curves
net.plot_train_curves()

Epoch 1/10 - loss: 5732.5180 - accuracy: 0.8540
Epoch 2/10 - loss: 9350.2387 - accuracy: 0.8673
Epoch 3/10 - loss: 18636.8414 - accuracy: 0.7767


C:\Users\mrjoa\Desktop\Uni\2 Semestre\SI\Apof\Trabalho_1\AProf_TP1\Code\RNN\layers\sigmoid.py:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-input))


Epoch 4/10 - loss: 22665.6592 - accuracy: 0.7533
Epoch 5/10 - loss: 17657.8700 - accuracy: 0.8007
Epoch 6/10 - loss: 22555.5300 - accuracy: 0.7663
Epoch 7/10 - loss: 18084.4018 - accuracy: 0.8117
Epoch 8/10 - loss: 13231.1960 - accuracy: 0.8610
Epoch 9/10 - loss: 15021.6551 - accuracy: 0.8430
Epoch 10/10 - loss: 10623.7952 - accuracy: 0.8903


In [6]:
# Test network

out = net.predict(X_test)
out = out.reshape(out.shape[1], 1)

if y_test is not None:
    print(net.score(y_test, out))

ValueError: cannot reshape array of size 1000 into shape (1,1)

In [7]:
# Store results

store_results = './Results/rnn_results.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(store_results), exist_ok=True)

results = dataset.merge_results(ids, out)
results.to_csv(store_results, sep='\t', index=False)

In [8]:
# Store network model

model_filepath = './Model/rnn'
model_key = 'rnn'

store_model.store_model(model_filepath, model_key, net)

In [9]:
# Retrieve network model

model_filepath = './Model/rnn'
model_key = 'rnn'

net = store_model.retrieve_model(model_filepath, model_key)